#         Lecture 13                    
                                            
## Advanced topics with linear regression      
   - Multiple regression                   
   - Model choice with multiple regressors 
   - Prediction with multiple regressors:  
        - useful graphs for predictions     
        - Confidence Interval               
        - Prediction Interval               
    - Robustness tests/External validity    
        - Time/Location/Type                
    - Extra: training and test sample       
                                            
#### Case Study:                                 
 - Hotels Europe                        
                                            
#### Dataset used:                               
 - hotel-europe 
 
 ___

Import packages

In [ ]:
import pandas as pd
import numpy as np
from plotnine import *
import statsmodels.formula.api as smf
from stargazer.stargazer import Stargazer

%matplotlib inline

Import Data:\
    now we use all the observations from Europe:

In [ ]:
hotels_europe_price = pd.read_csv("https://osf.io/p6tyr/download")
hotels_europe_features = pd.read_csv("https://osf.io/utwjs/download")

Join them by hotel_id

In [ ]:
europe = hotels_europe_price.merge(hotels_europe_features, on = "hotel_id")
europe.head()

In [ ]:
del hotels_europe_price
del hotels_europe_features

### Multiple Linear Regression

#### I) Predict Hotel prices in Vienna with multiple regressors!

##### A) Data management:

a) get the needed sample:

In [ ]:
vienna = (
    europe.loc[lambda x: x["accommodation_type"] == "Hotel"]
    .loc[lambda x: (x["year"] == 2017) & (x["month"] == 11) & (x["weekend"] == 0)]
    .loc[lambda x: x["city_actual"] == "Vienna"]
    .loc[lambda x: (x["stars"] >= 3) & (x["stars"] <= 4)]
    .loc[lambda x: x["stars"].notnull()]
    .loc[lambda x: x["price"]<=600]
)

 b) calculate log price

In [ ]:
vienna["lnprice"] = np.log(vienna["price"])

2) Quick reminder: check the descriptives + association

Summary statistics on price and log of price

In [ ]:
(
    vienna.filter(["price", "lnprice", "distance", "stars", "rating"])
    .describe(percentiles=[0.5, 0.95])
    .T
)

Look at the scatter plots - with the visual inspection we can decide which transformation to use!\
p1: distance vs price

In [ ]:
(
    ggplot(vienna, aes(x="distance", y="lnprice"))
    + geom_point(color="red", size=2)
    + geom_smooth(method="loess", color = "blue")
    + labs(x="Distance to city center (miles)", y="Log of price (US dollars)")
    + theme_bw()
)

 Log for price, Distance: check linear spline with knots at 1 and 4

Stars vs price

In [ ]:
(
    ggplot(vienna, aes(x="stars", y="lnprice"))
    + geom_point(color="red", size=2)
    + geom_smooth(method="lm", color="blue")
    + labs(x="Star of the Hotel", y="Log of price (US dollars)")
    + theme_bw()
)

Star of a hotel is discrete value: may use as it is, but using it as dummies would make our model more flexible!


#### Task: 
plot p3: rating vs price\
    What can you infer?

In [ ]:
(
    ggplot(vienna, aes(x="rating", y="lnprice"))
    + geom_point(color="red", size=2)
    + geom_smooth(method="loess", color = "blue")
    + labs(x = "Ratings of the hotel",y = "Log of price (US dollars)")
    + theme_bw()
)

 Rating: check with simple linear and with linear spline around value of 3.5

#### Notes:
  - when the outcome variable is (much) better for log transformation (for most of the variables):
     then you will need to use it in your model, there is nothing to do
  - however if the regressor is needed then it is enough to decide for that particular variable only!

#### II) Running regressions:

Baseline A: use only rating with heteroscedastic SE

In [ ]:
reg0 = smf.ols("lnprice ~ rating", data=vienna).fit(cov_type ="HC3")
print(reg0.summary())

 Baseline B: use only distance with heteroscedastic SE

In [ ]:
reg1 = smf.ols("lnprice ~ distance", data=vienna).fit(cov_type ="HC3")
print(reg1.summary())

Multiple regression with both rating and distance

In [ ]:
reg2 = smf.ols("lnprice ~ distance + rating", data=vienna).fit(cov_type ="HC3")
print(reg2.summary())

Add the number of stars to out model:\
As stars are discrete values: better to use `dummy` variables instead of one 'quasi-continuous' variable


In [ ]:
vienna["star3"] = np.where(vienna["stars"] == 3,1,0)
vienna["star35"] = np.where(vienna["stars"] == 3.5,1,0)
vienna["star4"] = np.where(vienna["stars"] == 4,1,0)

#### Task:
Add stars as dummies to the model with heteroscedastic SE\
Discuss what the coefficients mean for star3 and star35! What about star4?

In [ ]:
reg3 = smf.ols("lnprice ~ distance + rating + star3 + star35", data=vienna).fit(cov_type ="HC3")
reg3.summary()

Compare results

In [ ]:
table = Stargazer([reg0, reg1, reg2,reg3])
table

More complex models: this is the art of our profession: find the good knot points 
 
 (again this is why we do scatter plots)

Note, Python does not have an `lnspline` function as R, so we wrote one


In [ ]:
import copy
def lspline(series, knots):
    def knot_ceil(vector, knot):
        vector_copy = copy.deepcopy(vector)
        vector_copy[vector_copy > knot] = knot
        return vector_copy

    if type(knots) != list:
        knots = [knots]
    design_matrix = None
    vector = series.values

    for i in range(len(knots)):
        # print(i)
        # print(vector)
        if i == 0:
            column = knot_ceil(vector, knots[i])
        else:
            column = knot_ceil(vector, knots[i] - knots[i - 1])
        # print(column)
        if i == 0:
            design_matrix = column
        else:
            design_matrix = np.column_stack((design_matrix, column))
        # print(design_matrix)
        vector = vector - column
    design_matrix = np.column_stack((design_matrix, vector))
    # print(design_matrix)
    return design_matrix

In [ ]:
reg4 = smf.ols(
    "lnprice ~ lspline(distance, [1,4]) + lspline(rating, 3.5) + star3 + star35",
    data=vienna,
).fit(cov_type="HC3")
reg4.summary()

In [ ]:
table = Stargazer([reg2,reg3, reg4])
table

### III) Analyzing the results: our choice is regression 4

Save the predicted and residual values

In [ ]:
vienna["lnprice_hat"] = reg4.fittedvalues
vienna["lnprice_resid"] = reg4.resid


Note we are interested in real price values not in logs:\
the first term is straight forward, however the second exp( reg4$ssr /2 ) is a correction term 
  which is needed when dealing with log to exp transformation. Later in DA3 it will be discussed.

In [ ]:
corr_term = np.exp(vienna["lnprice_resid"].mean() / 2)
vienna["price_hat"] = np.exp(vienna["lnprice_hat"]) * corr_term

List of 5 best deals

In [ ]:
vienna.sort_values(by="lnprice_resid").head(5).filter(
    [
        "hotel_id",
        "price",
        "price_hat",
        "lnprice",
        "lnprice_hat",
        "lnprice_resid",
        "distance",
        "stars",
        "rating",
    ]
)

Why the 5 best lnprice_resid is the same as if we have used price_hat? Discuss!

### Two useful graphs:

 1) y - yhat graph (regression line must be the same as the 45 degree line!)

In [ ]:
(
    ggplot(vienna, aes(x="lnprice_hat", y="lnprice"))
    + geom_point()
    + geom_smooth(method="lm", formula="y~x", se=False)
    + labs(x="ln(predicted price, US dollars) ", y="ln(price, US dollars)")
    + geom_segment(
        aes(x=4.8, y=4.1, xend=4.68, yend=4.1),
        arrow=arrow(),
        color="red",
    )
    + annotate("text", x=4.95, y=4.1, label="Best deal", size=8, color="red")
    + geom_abline(intercept=0, slope=1, size=0.5, color="red", linetype="dashed")
    + coord_cartesian(xlim=(4, 5.5), ylim=(4, 5.5))
    + theme_bw()
)

 2) residual - yhat graph: it needs to be flat

In [ ]:
(
    ggplot(vienna, aes(x="lnprice_hat", y="lnprice_resid"))
    + geom_point(color="red", size=2)
    + geom_smooth(method="lm", colour="blue", se=False, formula="y~x")
    + labs(x="ln(Predicted hotel price, US dollars)", y="Residuals")
    + theme_bw()
)

Helps to evaluate where we tend to make larger errors. It can be used for different regressors!

### Task:

check the graphs across distance! (note it is not always flat, discuss, why!)\
residual - distance graph 

In [ ]:
(
    ggplot(vienna, aes(x="distance", y="lnprice_resid"))
    + geom_point(color="red", size=2)
    + geom_smooth(method="lm", colour="blue", se=False, formula="y~x")
    + labs(x="Distance from city center (miles)", y="Residuals")
    + theme_bw()
)

### Confidence interval for the E(Y|X):
1) predict the outcomes with predict command and use the se.fit = T, \
    this will give you the standard errors for the conditional expectation!


In [ ]:
pred_CI = reg4.get_prediction().summary_frame(alpha=0.05)
pred_CI.head()

Add the CI values to vienna dataset

In [ ]:
vienna["CI_up"] = pred_CI["mean_ci_lower"]
vienna["CI_low"] = pred_CI["mean_ci_upper"]

 Why we usually do not use such graphs, when evaluating multiple regression results:

In [ ]:
(
    ggplot(data=vienna)
    + geom_point(aes(x="distance", y="lnprice"), color="blue", size=2)
    + geom_line(aes(x="distance", y="lnprice_hat"), color="red", size=1)
    + geom_line(aes(x="distance", y="CI_up"), color="red", size=0.5, linetype="dashed")
    + geom_line(aes(x="distance", y="CI_low"), color="red", size=0.5, linetype="dashed")
    + labs(x="Distance to city center (miles)", y="Log of price (US dollars)")
    + theme_bw()
)

### Price a new hotel with the model

However you can predict any (new) potential variable

In [ ]:
new_hotel_vienna = pd.DataFrame(
    {"distance": 2.5, "star3": 0, "star35": 0, "rating": 3.2}, index=[0]
)

pred_new = reg4.get_prediction(new_hotel_vienna).summary_frame(alpha=0.05)
pred_new

Note: you are not really looking for log-price, hence the correction

In [ ]:
pred_new["pred_price"] = np.exp(pred_new["mean"])*corr_term
pred_new["CI_low"] = np.exp(pred_new["mean_ci_lower"])*corr_term
pred_new["CI_up"] = np.exp(pred_new["mean_ci_upper"])*corr_term

pred_new.filter(["CI_low","pred_price","CI_up"])

Is the Confidence Interval symmetric? In which case? Why? Discuss! 

### Prediction interval: considers the inherent error as well!

Note: in statsmodels, mean_ci (lower, upper) is CI, and obs_ci (lower, upper) is the prediction interval

In [ ]:
pred_new["PI_low"] = np.exp(pred_new["obs_ci_lower"])*corr_term
pred_new["PI_up"] = np.exp(pred_new["obs_ci_upper"])*corr_term

pred_new.filter(["PI_low","pred_price","PI_up"])

 Let us compare the two results for our newly predicted hotel:

In [ ]:
pred_new.filter(["pred_price","CI_low","CI_up","PI_low","PI_up"])

 Discuss what the results mean! How would you price this hotel?

### Task:

There is a new hotel with the following feature values that you would like to price \

    distance = 0.25 , star3 = 0 , star35 = 0, rating = 4.1
   
- Now, you use polynomials rather than piecewise linear spline up to 3rd order for distance and rating. 
- Use dummies for stars and use heteroskedastic robust SE!
- Estimate the model, and check the distance and ratings against the residuals for the LEVELS.
- Calculate the CI and PI with 80% levels, using the `level` input argument for this hotel based on the model.

What price would you use? Argue!

Model:

In [ ]:
reg_poly = smf.ols(
    "lnprice ~ distance + np.power(distance, 2)+ np.power(distance, 3) + rating + np.power(rating, 2)+ np.power(rating, 3)+ star3 + star35",
    data=vienna,
).fit(cov_type="HC3")

Predictions and errors 

1. for log values

In [ ]:
vienna["lnprice_hat_rp"] = reg_poly.fittedvalues
vienna["lnprice_resid_rp"] = reg_poly.resid

2.  for the levels

In [ ]:
corr_term_rp = np.exp(vienna["lnprice_resid_rp"].mean() / 2)
vienna["price_hat_rp"] = np.exp(vienna["lnprice_hat_rp"]) * corr_term_rp
vienna["price_resid_rp"] = vienna["price"] - vienna["price_hat_rp"]

Create a residual vs distance graph

In [ ]:
(
    ggplot(vienna, aes(x="distance", y="price_resid_rp"))
    + geom_point(color="red", size=2)
    + geom_smooth(method="lm", colour="blue", se=False, formula="y~x")
    + labs(x="Distance", y="Residuals")
    + theme_bw()
)

Create a residual vs ratings graph 

-> model makes large errors around 4,   in both directions, but mainly underprice the hotels. Not too serious.

In [ ]:
(
    ggplot(vienna, aes(x="rating", y="price_resid_rp"))
    + geom_point(color="red", size=2)
    + geom_smooth(method="lm", colour="blue", se=False, formula="y~x")
    + labs(x="Ratings of the hotels", y="Residuals")
    + scale_x_continuous(
        expand=(0.01, 0.01), limits=(2, 5), breaks=np.arange(2, 5, 0.5)
    )
    + theme_bw()
)

#### CI and PI for the new hotel

In [ ]:
new_hotel2_vienna = pd.DataFrame(
    {"distance": 0.25, "star3": 0, "star35": 0, "rating": 4.1}, index=[0]
)

pred_new2 = reg_poly.get_prediction(new_hotel2_vienna).summary_frame(alpha=0.2)
pred_new2

In [ ]:
pred_new2["pred_price"] = np.exp(pred_new2["mean"])*corr_term_rp
pred_new2["CI_low"] = np.exp(pred_new2["mean_ci_lower"])*corr_term_rp
pred_new2["CI_up"] = np.exp(pred_new2["mean_ci_upper"])*corr_term_rp
pred_new2["PI_low"] = np.exp(pred_new2["obs_ci_lower"])*corr_term_rp
pred_new2["PI_up"] = np.exp(pred_new2["obs_ci_upper"])*corr_term_rp
pred_new2.filter(["pred_price","CI_low","CI_up","PI_low","PI_up"]).round(2)

As the model probably underprice this hotel around this distance rating, \
  I would use a larger price than predicted. However, this is a new hotel, \
  thus start with a lower price make some good word-of-mouth can be beneficial.\
 In sum something around the CI_high can be a good point if there are no other reasons.

## IV) External validity of the model


 We want to test how our favorite model would perform if we change:\
 time OR place OR type of observations (e.g. Apartment instead of Hotels)

0) Add the variable transformations to the general dataset: hotels-europe

In [ ]:
europe["lnprice"] = np.log(europe["price"])
europe["star3"] = np.where(europe["stars"] == 3, 1, 0)
europe["star35"] = np.where(europe["stars"] == 3.5, 1, 0)

 ### 1) First let check for different time: Vienna multiple time

In [ ]:
vienna_m_time = (
    europe.loc[lambda x: x["accommodation_type"] == "Hotel"]
    .loc[lambda x: x["city_actual"] == "Vienna"]
    .loc[lambda x: x["nnights"] == 1]
    .loc[lambda x: (x["stars"] >= 3) & (x["stars"] <= 4)]
    .loc[lambda x: x["stars"].notnull()]
    .loc[lambda x: x["price"] <= 600]
)

IMPORTANT: for a fair comparison, we need to use the SAME HOTELS!\
we have three alternative candidates:
    
    main: 2017/11 weekday
    1): 2017/11 on a weekend
    2): 2017 December, holiday
    3): 2018 June, weekend

first we get our preferred dates

In [ ]:
vienna_m_time = vienna_m_time.loc[
    lambda x: ((x["year"] == 2017) & (x["month"] == 11) & (x["weekend"] == 0))
    | ((x["year"] == 2017) & (x["month"] == 11) & (x["weekend"] == 1))
    | ((x["year"] == 2017) & (x["month"] == 12) & (x["holiday"] == 1))
    | ((x["year"] == 2018) & (x["month"] == 6) & (x["weekend"] == 1))
]

Secondly we extract the frequencies for each hotels

In [ ]:
vienna_m_time["hotel_id"].value_counts()

Hotels with frequency of four are in all of our dates

In [ ]:
four_freq_id = vienna_m_time["hotel_id"].value_counts().loc[lambda x: x == 4].index

vienna_m_time = vienna_m_time.loc[lambda x: x["hotel_id"].isin(four_freq_id)]

Save the formula for the main model:

In [ ]:
m_form = "lnprice ~ lspline(distance, [1,4]) + lspline(rating, 3.5) + star3 + star35"

Our main model: 2017/11 in a weekday

In [ ]:
regt_0 = smf.ols(
    m_form,
    data=vienna_m_time.loc[
        lambda x: ((x["year"] == 2017) & (x["month"] == 11) & (x["weekend"] == 0))
    ],
).fit(cov_type="HC3")

 Alternatively 1) 2017/11 on a weekend

In [ ]:
regt_1 = smf.ols(
    m_form,
    data=vienna_m_time.loc[
        lambda x: ((x["year"] == 2017) & (x["month"] == 11) & (x["weekend"] == 1))
    ],
).fit(cov_type="HC3")

Alternatively 2) 2017 December, holiday

In [ ]:
regt_2 = smf.ols(
    m_form,
    data=vienna_m_time.loc[
        lambda x: ((x["year"] == 2017) & (x["month"] == 12) & (x["holiday"] == 1))
    ],
).fit(cov_type="HC3")

Alternatively 2) 2018 June, weekend

In [ ]:
regt_3 = smf.ols(
    m_form,
    data=vienna_m_time.loc[
        lambda x: ((x["year"] == 2018) & (x["month"] == 6) & (x["weekend"] == 1))
    ],
).fit(cov_type="HC3")

Compare the results:

In [ ]:
table = Stargazer([regt_0, regt_1, regt_2, regt_3])
table.custom_columns(
    [
        "2017/11 weekday",
        "2017/11 on a weekend",
        "2017 December, holiday",
        "2018 June, weekend",
    ],
    [1, 1, 1, 1],
)
table

With prediction look at: R2, and stability of the parameters!

### 2) Second let's check for different accommodation types in Vienna:

Compare hotels with Apartments:

Note: we only change one thing at a time!

In [ ]:
vienna_h_vs_a = (
    europe.loc[
        lambda x: (x["accommodation_type"] == "Hotel")
        | (x["accommodation_type"] == "Apartment")
    ]
    .loc[lambda x: (x["year"] == 2017) & (x["month"] == 11) & (x["weekend"] == 1)]
    .loc[lambda x: x["city_actual"] == "Vienna"]
    .loc[lambda x: x["nnights"] == 1]
    .loc[lambda x: (x["stars"] >= 3) & (x["stars"] <= 4)]
    .loc[lambda x: x["stars"].notnull()]
    .loc[lambda x: x["price"] <= 600]
    .loc[lambda x: x["rating"].notnull()]
)

Note: here we can not compare the same observations as they are inherently different

Run regression for the hotels

In [ ]:
regh = smf.ols(
    m_form, data=vienna_h_vs_a.loc[lambda x: x["accommodation_type"] == "Hotel"]
).fit(cov_type="HC3")

Run regression for the apartments

In [ ]:
rega = smf.ols(
    m_form, data=vienna_h_vs_a.loc[lambda x: x["accommodation_type"] == "Apartment"]
).fit(cov_type="HC3")

In [ ]:
table = Stargazer([regh, rega])
table.custom_columns(["Hotels", "Apartments"], [1, 1])
table

 ### Task: 
    
   3) Compare different cities:

Check Vienna, Amsterdam, and Barcelona!


Note: Get rid of the price filter!


In [ ]:
hotels_cities = (
    europe.loc[lambda x: x["accommodation_type"] == "Hotel"]
    .loc[lambda x: (x["year"] == 2017) & (x["month"] == 11) & (x["weekend"] == 1)]
    .loc[
        lambda x: (x["city_actual"] == "Vienna")
        | (x["city_actual"] == "Amsterdam")
        | (x["city_actual"] == "Barcelona")
    ]
    .loc[lambda x: x["nnights"] == 1]
    .loc[lambda x: (x["stars"] >= 3) & (x["stars"] <= 4)]
    .loc[lambda x: x["stars"].notnull()]
    .loc[lambda x: x["rating"].notnull()]
)

Run regression for Vienna

In [ ]:
reg_v = smf.ols(
    m_form, data=hotels_cities.loc[lambda x: x["city_actual"] == "Vienna"]
).fit(cov_type="HC3")

Run regression for Amsterdam

In [ ]:
reg_a = smf.ols(
    m_form, data=hotels_cities.loc[lambda x: x["city_actual"] == "Amsterdam"]
).fit(cov_type="HC3")

Run regression for Barcelona

In [ ]:
reg_b = smf.ols(
    m_form, data=hotels_cities.loc[lambda x: x["city_actual"] == "Barcelona"]
).fit(cov_type="HC3")

In [ ]:
table = Stargazer([reg_v, reg_a, reg_b])
table.custom_columns(["Vienna", "Amsterdam", "Barcelona"], [1, 1, 1])
table

### Extra:
 In prediction competitions usually there is a training/available sample\
   and a test/locked sample for the competitors. \
   The task is to build a model which gives the best prediction (according to a pre-defined measure)
   
 E.g. uses our original sample, and create a train-test split using the sklean package

In [ ]:
from sklearn.model_selection import train_test_split

vienna_train, vienna_test = train_test_split(vienna, train_size=150, random_state=234)

Let us use two competing models:

model 1: simple multivariate model


In [ ]:
pred_m1 = smf.ols(
    "lnprice ~ distance + rating + star3 + star35", data=vienna_train
).fit()

model 2: multivariate model with linear splines

In [ ]:
pred_m2 = smf.ols(m_form, data=vienna_train).fit()

Now let us use these models on our test sample to predict the values

In [ ]:
pred_compare = pd.DataFrame(
    [
        pred_m1.predict(vienna_test),
        pred_m2.predict(vienna_test),
        vienna_test["lnprice"],
    ],
    index=["m1_lnprice", "m2_lnprice", "actual_lnprice"],
).T

If you want to visualize

In [ ]:
pred_compare.shape

In [ ]:
(
    ggplot(pred_compare)
    + geom_point(aes(x="actual_lnprice", y="m1_lnprice"), color="red", size=2)
    + geom_point(aes(x="actual_lnprice", y="m2_lnprice"), color="blue", size=2)
    + geom_line(aes(x="actual_lnprice", y="actual_lnprice"), color="black", size=1)
    + labs(x="Actual log price", y="Predicted log prices")
    + coord_cartesian(xlim=(3.7, 5.5), ylim=(3.7, 5.5))
    + theme_bw()
)

Evaluate according to RMSE measure:

In [ ]:
from sklearn.metrics import mean_squared_error

The two rmse values

In [ ]:
mean_squared_error(
    pred_compare["actual_lnprice"], pred_compare["m1_lnprice"], squared=False
)

In [ ]:
mean_squared_error(
    pred_compare["actual_lnprice"], pred_compare["m2_lnprice"], squared=False
)

Therefore model 2 wins this competition according to RMSE measure.